In [594]:
import requests
import numpy as np
import pandas as pd
import psycopg2
from bs4 import BeautifulSoup
from datetime import datetime, date, time
import string
import time
from dateutil import parser
import geopy.distance
import re
import pytz    
import tzlocal 
import matplotlib.pyplot as plt
import calendar
import matplotlib
import re
pd.options.display.max_columns=100
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')


<input type="hidden" id="captchaStr" name="captchaStr" value="" />
<input type="hidden" id="type" name="type" value="igc" />

<input type="hidden" id="file" name="file" value="" />

### HOW TO SCRAPE A LOG FILE

In [ ]:
data = {"flightID": 714360, "file":"", "type":"igc", "captchaStr":""}

In [ ]:
response = requests.post('http://www.paraglidingforum.com/leonardo/download_igc.php', data = data)
response.status_code

In [ ]:
response.text

### TRYING TO GET PILOT NAME

In [472]:
def strip_name(string):
    step1 = string.split('title')[-1]
    step2 = re.findall('>(\D[^<][^\n]{,30})<',step1)
    return step2

In [484]:
pilot_list = {}

for page in range(1,110):

    headers = {
        'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36',
        'Referer': 'http://www.paraglidingforum.com/leonardo/page/filter',
        'page_num': str(page)}
    response = requests.get('http://www.paraglidingforum.com/leonardo/tracks/world/alltimes/brand:all,cat:0,class:all,xctype:all,club:all,pilot:0_0,takeoff:all&fltr=428929_42145.&page_num={}'.format(page),headers = headers)
    soup = BeautifulSoup(response.text, "html.parser")

    flight_numbers = [link["href"].split('/')[3] for link in soup.find_all("a", {"class":"flightLink"})]
    list_of_classes = [str(i) for i in soup.find_all("div", {"class":"pilotLink"})]
    pilot_names = [strip_name(str(i)) for i in  soup.find_all("div", {"class":"pilotLink"})]
    for a,b in zip(flight_numbers,pilot_names):
        if b==[]:
            pilot_list[a]='UNKNOWN'
        else:
            pilot_list[a]=b[0]
    print('page {} done'.format(page))

page 1 done
page 2 done
page 3 done
page 4 done
page 5 done
page 6 done
page 7 done
page 8 done
page 9 done
page 10 done
page 11 done
page 12 done
page 13 done
page 14 done
page 15 done
page 16 done
page 17 done
page 18 done
page 19 done
page 20 done
page 21 done
page 22 done
page 23 done
page 24 done
page 25 done
page 26 done
page 27 done
page 28 done
page 29 done
page 30 done
page 31 done
page 32 done
page 33 done
page 34 done
page 35 done
page 36 done
page 37 done
page 38 done
page 39 done
page 40 done
page 41 done
page 42 done
page 43 done
page 44 done
page 45 done
page 46 done
page 47 done
page 48 done
page 49 done
page 50 done
page 51 done
page 52 done
page 53 done
page 54 done
page 55 done
page 56 done
page 57 done
page 58 done
page 59 done
page 60 done
page 61 done
page 62 done
page 63 done
page 64 done
page 65 done
page 66 done
page 67 done
page 68 done
page 69 done
page 70 done
page 71 done
page 72 done
page 73 done
page 74 done
page 75 done
page 76 done
page 77 done
page 78 

In [482]:
 for i in pilot_list.items():
        if i[1]==[]:
            pilot_list[i[0]]='UNKNOWN'
        else:
            pilot_list[i[0]]=i[0]

In [637]:
# pilot_list

### HOW TO SCRAPE THE FLIGHT NUMBERS

In [ ]:

flight_number = []

for page in range(1,64):

    headers = {
        'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36',
        'Referer': 'http://www.paraglidingforum.com/leonardo/page/filter',
        'page_num': str(page)}
    response = requests.get('http://www.paraglidingforum.com/leonardo/tracks/world/alltimes/brand:all,cat:0,class:all,xctype:all,club:all,pilot:0_0,takeoff:all&fltr=2040E0004242145_8929.&page_num={}'.format(page),headers = headers)
    soup = BeautifulSoup(response.text, "html.parser")
    flight_number.append([link["href"].split('/')[3] for link in soup.find_all("a", {"class":"flightLink"})])

In [ ]:
final_list = []
for page in flight_number:
    for flight in page:
        final_list.append(flight)

In [ ]:
#saving the flight numbers in a txt file

outfile = open('flights2.txt', 'w')

for item in final_list:
    outfile.write(item)
    outfile.write(',')

### INTEGRATING WITH THE POSTGRES DB

In [ ]:
conn = psycopg2.connect("dbname = soaring_predictor")

In [ ]:
cur = conn.cursor()

In [ ]:
query = """
INSERT INTO flight_log 
(flight,log) 
VALUES(%s,%s);
"""

test = """
SELECT log
FROM flight_log
where flight = 1360169;"""


In [ ]:
#testing the conmection 
cur.execute(test)
cur.fetchone()

In [ ]:
test_list = final_list[:5]
test_list

In [ ]:
## This loop scrapes & saves into the SQL DB (1:1)

good_char = set(string.printable)

for i,flight in enumerate(test_list):
    
    print('Trying flight {}'.format(flight))
    data = {"flightID": int(flight), "file":"", "type":"igc", "captchaStr":""}
    response = requests.post('http://www.paraglidingforum.com/leonardo/download_igc.php', data = data)
    time.sleep(1)
    output = response.text


    data2 = [(int(flight),output)]


    records_list_template = ','.join(['%s'] * len(data2))
    insert_query = 'insert into flight_log (flight, log) values {}'.format(records_list_template)
    try:
        cur.execute(insert_query, data2)
    
    except:
        # This line does not do anything other than avoid an error (it was a try to take bad chars from the log)
        output = "".join([letter for letter in output if letter in good_char])
        print('Flight {} failed'.format(i))
        
   
   if i % 100 == 0:
        conn.commit()
        print('Saved {} at {}'.format(i, datetime.now().strftime('%H:%M')))

conn.commit()


In [ ]:
cur.close()  # Close the cursor
conn.close() # Close the connection

### FUNCTIONS TO GET FEATURES FROM THE FLIGHT LOGS

In [622]:
'''List of features I want to get from the flight log:
            DATE
            PILOT NAME
            LAUNCH TIME
            LANDING TIME
            TOTAL FLIGHT TIME
            MAX ALTITUDE
            TOTAL ALTITUDE GAIN
            LAUNCH COORDINATES
            LANDING COORDINATES
            EUCLIDIAN DISTANCE BETWEEN LAUNCH AND LANDING
            LANDING = LAUNCH
            LEFT PERIMETER
'''

#returns the date of the flight (as datetime object)

def get_date(igc_file):
#     return parser.parse(igc_file.split('\n')[1][-6:],dayfirst=True).date()
    if igc_file.find('HFDTEDATE:') == -1:
        s = igc_file.find('HFDTE') + 5
    else:
        s = igc_file.find('HFDTEDATE:') + 10
    return parser.parse(igc_file[s:s+6],dayfirst=True).date()

# returns the month (as a string, in the format JAN, FEB...)

def get_month(dt_obj):
#     return calendar.month_name[dt_obj.month][0:3].upper()
    return dt_obj.month

def get_year(dt_obj):
#     return calendar.month_name[dt_obj.month][0:3].upper()
    return dt_obj.year



#returns pilot name

def pilot_name(igc_file):
    if igc_file.find('PILOT') == -1:
        return 'UNKNOWN'
    else:
        p = igc_file.find('PILOT')
        s = igc_file.find(':',p)
        e = igc_file.find('\n',s)
        return igc_file[s+1:e].strip('\r').upper().split(' ')[-1]
        
# returns launch time (as datetime object)

# This is to adjust the time from UTC tp PST

local_timezone = tzlocal.get_localzone() # get pytz tzinfo
utc_time = datetime.strptime("2011-01-21 02:37:21", "%Y-%m-%d %H:%M:%S")
local_time = utc_time.replace(tzinfo=pytz.utc).astimezone(local_timezone)

def launch_time(igc_file):
    m = re.search('B(.{6})47', igc_file, )
    if m:
        found = m.group(1)
        return datetime.strptime(found, '%H%M%S').replace(tzinfo=pytz.utc).astimezone(local_timezone).time()
    
# returns landing time (as datetime object)

def landing_time(igc_file):
    m = re.findall('B([0123456789]{13})N', igc_file )
    found = m[-1][:6]
    return datetime.strptime(found, '%H%M%S').replace(tzinfo=pytz.utc).astimezone(local_timezone).time()



# returns the total flying time (as datatime object, in seconds)

def flying_time(igc_file):
    x = launch_time(igc_file)
    y = landing_time(igc_file)
    diff = datetime.combine(date.today(), y) - datetime.combine(date.today(), x)
    return (diff.total_seconds() / 60 )  
   

#strip the altitude from a string - returns an int

def get_alt(alt_str):
    if len(alt_str)<=4:
        return int(alt_str)
    else:
        alt_str1 = int(alt_str[0:5].strip('-'))
        alt_str2 = int(alt_str[5:10].strip('-'))
        if alt_str2 != 0:
            return alt_str2
        elif alt_str !=0:
            return alt_str1

# for debug purposes ony - returns list of strings
def find_alt_entry(igc_file):
    a = re.search('WA([-0123456789]{3,16})', igc_file)
    if a:
        return a.group(1)
    

# returns the max altitude in the fligh (in meters)

def max_alt(igc_file):
    m = re.findall('WA([-0123456789]{2,16})', igc_file)
    return max([get_alt(i) for i in m])
    



# returns the total altitude gain in the flight (in meters):

def total_alt_gain(igc_file):
    total_alt_gain = 0
    m = re.findall('WA([-0123456789]{3,16})', igc_file)
    for s, t in zip(m, m[1:]):
        try:
            if len(t) <= 4:
                if int(t) > int(s):
                    total_alt_gain += (int(t) - int(s))
            elif int(t[5:10]) > int(s[5:10]):
                total_alt_gain += (int(t[5:10]) - int(s[5:10]))
        except:
            next
    return total_alt_gain

    
    

# returns launch coordinates

def launch_coord(igc_file):
    m = re.search('B(.{22}[01234567890NW])A', igc_file )
    if m:
        found = m.group(1)
        e = found.find('N')
        lat = found[e-7:e+1]
        lon = found[e+1:]
        lat = dec_lat(lat)
        lon = dec_lon(lon)
        return (lat,lon)

# returns landing coordinates

def landing_coord(igc_file):
    m = re.findall('B([01234567890NW]{23})A', igc_file )
    found = m[-1]
    e = found.find('N')
    lat = found[e-7:e+1]
    lon = found[e+1:]
    lat = dec_lat(lat)
    lon = dec_lon(lon)
    return (lat,lon)  


# returns the coordinates in decimals from string

def get_coord(item):
    lat = dec_lat(item[8:16])
    lon = dec_lon(item[16:25])
    return (lat,lon)

# returns the latitude in decimals from segregated string


def dec_lat(lat):
    D = float(lat[0:2])
    M = float(lat[2:7])
    S = float(lat[4:7])
    DD = D + M/60000 
    if lat[-1] == 'N':
        lat = DD
    elif lat[-1] == 'S':
        lat = -DD
    return lat

# returns the longitude in decimals from segregated string

def dec_lon(lon):
    D = float(lon[0:3])
    M = float(lon[3:8])
    S = float(lon[5:8])
    DD = D + M/60000 
    if lon[-1] == 'E':
        lon = DD
    elif lon[-1] == 'W':
        lon = -DD
    return lon


# returns the euclidian distance between landing and launch

def final_eucl_distance(igc_file):
    return geopy.distance.vincenty(launch_coord(igc_file), landing_coord(igc_file)).km


# returns the euclidian distance between two points (entry in coordinates, returns distance in km)

def eucl_distance(pt1,pt2):
    return geopy.distance.vincenty(pt1, pt2).km

# checks if landed at TIger LZ (0 if not, 1 if yes)

def landed_lz(igc_file):
    NW_corner = (47.502,-122.026)
    SE_corner = (47.498, -122.018)
    land_coord = landing_coord(igc_file)
    return int(NW_corner[0] <= land_coord[0] <= SE_corner[0] and NW_corner[1] >= land_coord[1] >= SE_corner[1])


#Checks if left the perimeter past north or east of the ridge tower or south or west of the nudist club

def left_perimeter(igc_file):    
    
    NE_edge = (47.5092,-121.986)
    SW_edge = (47.488,-122.04)
    left_per = 0
    m = re.findall('B([01234567890NW]{23})A', igc_file )
    for record in m:
        e = record.find('N')
        lat = record[e-7:e+1]
        lon = record[e+1:]
        lat = dec_lat(lat)
        lon = dec_lon(lon)
        if  (lat >= NE_edge[0] or lat <= SW_edge[0] or lon >= NE_edge[1]  or lon <= SW_edge[1]):
            left_per = 1
            break
    return left_per  


# returns day of the year

def doy(dt_obj):
    return int(dt_obj.strftime('%j'))




### CLEANING AND CREATING LABEL FEATUTES

In [623]:
conn = psycopg2.connect("dbname = soaring_predictor")



In [624]:
Labels = pd.read_sql('SELECT * FROM flight_log;',conn)

In [625]:
'''There are some records with missing files...this list tags them. Next step is to drop them out'''

lst = []

for i in range(len(Labels)):
    try:
        get_date(Labels['log'][i])
    except:
        lst.append(i)
        
lst

[19,
 876,
 877,
 878,
 900,
 971,
 1012,
 1019,
 2085,
 2282,
 2994,
 2995,
 2996,
 3315,
 3359,
 3361,
 3833,
 3925,
 4059,
 4066,
 4084,
 4107,
 4108,
 4109,
 4112,
 4197,
 4259,
 4355,
 4436,
 4675]

In [626]:
Labels.drop(lst, inplace=True)

In [627]:
Labels.reset_index(inplace = True,drop=True)
Labels.columns

Index(['flight', 'log'], dtype='object')

In [628]:
# Just to check on corrupted files
fails = []
for i in range(len(Labels)):
    try:
        max_alt(Labels['log'][i])
    except:
        fails.append(i)
        
fails

[1048, 3096]

In [629]:
Labels.drop(fails, axis=0, inplace=True)
Labels.reset_index(inplace = True,drop=True)
Labels.columns
 

Index(['flight', 'log'], dtype='object')

In [642]:
'''Completed:
Labels['date'] = Labels['log'].apply(get_date)
Labels['pilot'] = Labels['log'].apply(pilot_name)
Labels['launch_time'] = Labels['log'].apply(launch_time)
Labels['landing_time']=Labels['log'].apply(landing_time)
Labels['flying_time'] = Labels['log'].apply(flying_time)
Labels['launch_coord']=Labels['log'].apply(launch_coord)
Labels['landing_coord']=Labels['log'].apply(landing_coord)
Labels['landed_lz']=Labels['log'].apply(landed_lz)
Labels['linear_dist']=Labels['log'].apply(final_eucl_distance)
Labels['max_alt']=Labels['log'].apply(max_alt)
Labels['total_alt_gain']=Labels['log'].apply(total_alt_gain)
Labels['left_perimeter']=Labels['log'].apply(left_perimeter)
Labels['month'] = Labels['date'].apply(get_month)
Labels['year'] = Labels['date'].apply(get_year)
--------------------------------------------------------
'''

# Labels['date'] = Labels['log'].apply(get_date)
# Labels['pilot'] = Labels['log'].apply(pilot_name)
# Labels['launch_time'] = Labels['log'].apply(launch_time)
# Labels['landing_time']=Labels['log'].apply(landing_time)
# Labels['flying_time'] = Labels['log'].apply(flying_time)
# Labels['launch_coord']=Labels['log'].apply(launch_coord)
# # Labels['landing_coord']=Labels['log'].apply(landing_coord)
# Labels['landed_lz']=Labels['log'].apply(landed_lz)
# Labels['linear_dist']=Labels['log'].apply(final_eucl_distance)
# Labels['max_alt']=Labels['log'].apply(max_alt)
# Labels['total_alt_gain']=Labels['log'].apply(total_alt_gain)
# Labels['left_perimeter']=Labels['log'].apply(left_perimeter)
# Labels['month'] = Labels['date'].apply(get_month)
# Labels['year'] = Labels['date'].apply(get_year)
# Labels['timestamp'] = pd.to_datetime(Labels['date'].apply(str) + ' '+ Labels['launch_time'].apply(str))
# Labels['doy']= Labels['date'].apply(doy)
Labels.head()

,flight,log,date,pilot,launch_time,landing_time,flying_time,launch_coord,landing_coord,landed_lz,linear_dist,max_alt,total_alt_gain,left_perimeter,month,year,timestamp,doy
0,1860369,AXGG1.66\r\nHFDTE010118\r\nHFFXA100\r\nHFPLTPI...,2018-01-01,,13:31:13,13:35:18,4.083333,"(47.50035, -122.00958333333334)","(47.5006, -122.02118333333334)",0,0.874419,517,0,0,1,2018,2018-01-01 13:31:13,1
1,1803029,AXCSAAA\nHFDTE211217\nHFFXA050\nHFPLTPILOTINCH...,2017-12-21,AARON,16:35:41,16:43:57,8.266667,"(47.49933333333333, -122.00848333333333)","(47.49993333333333, -122.02186666666667)",0,1.010559,548,83,0,12,2017,2017-12-21 16:35:41,355
2,1755461,AXGD Flymaster NavSD SN01525 SW1.06j\r\nHFDT...,2017-11-18,HENRIQUEZ,13:24:21,14:59:31,95.166667,"(47.49608333333333, -122.01065)","(47.50018333333333, -122.022)",0,0.969083,1057,2749,1,11,2017,2017-11-18 13:24:21,322
3,1757934,AXCSAAA\nHFDTE181117\nHFFXA050\nHFPLTPILOTINCH...,2017-11-18,MALOCSAY,13:26:17,14:21:06,54.816667,"(47.4967, -122.01033333333334)","(47.500366666666665, -122.02195)",0,0.965545,887,1123,1,11,2017,2017-11-18 13:26:17,322
4,1751790,AXCT XCTrack 0.6.0 on LGE Nexus 5X 8.0.0\r\nHP...,2017-11-17,LEONE,13:22:57,14:19:50,56.883333,"(47.49661666666667, -122.0102)","(47.4998, -122.02168333333333)",0,0.934814,1008,1446,1,11,2017,2017-11-17 13:22:57,321


In [647]:
Labels.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5076 entries, 0 to 5099
Data columns (total 18 columns):
flight            5076 non-null int64
log               5076 non-null object
date              5076 non-null object
pilot             5076 non-null object
launch_time       5076 non-null object
landing_time      5076 non-null object
flying_time       5076 non-null float64
launch_coord      5076 non-null object
landing_coord     5076 non-null object
landed_lz         5076 non-null int64
linear_dist       5076 non-null float64
max_alt           5076 non-null int64
total_alt_gain    5076 non-null int64
left_perimeter    5076 non-null int64
month             5076 non-null int64
year              5076 non-null int64
timestamp         5076 non-null datetime64[ns]
doy               5076 non-null int64
dtypes: datetime64[ns](1), float64(2), int64(8), object(7)
memory usage: 753.5+ KB


In [612]:
# DO NOT RUN THIS
# Labels['landed_lz'] = Labels['landed_lz'] == 0
# Labels['landed_lz'] = Labels['landed_lz'].astype(int)

In [645]:
Labels = Labels[Labels['flying_time']>0] # deletes rows with negative flying times

# Labels = Labels[Labels['year']>2012] # deletes rows older than 2013

In [646]:
#Removes records with crazy altitude - flight log is corrupted
l =list(Labels['flight'][Labels['max_alt']> 40000].index)
Labels.drop(l,inplace = True, axis=0)

In [618]:
# dropping a file with crazy max_alt
# Labels[Labels['max_alt']>3000]
# Labels.drop(3011, axis=0, inplace=True)
# Labels.reset_index(inplace = True, drop=True)


In [648]:
# dropping values with max_alt<200
Labels = Labels[Labels['max_alt']>500]

### Adjusting Pilot Names


In [649]:
def adjust_name(flight_number):
    return pilot_list[str(flight_number)]

In [654]:
Labels['pilot'] = Labels['flight'].apply(adjust_name)

In [655]:
Labels.columns

Index(['flight', 'log', 'date', 'pilot', 'launch_time', 'landing_time',
       'flying_time', 'launch_coord', 'landing_coord', 'landed_lz',
       'linear_dist', 'max_alt', 'total_alt_gain', 'left_perimeter', 'month',
       'year', 'timestamp', 'doy'],
      dtype='object')

In [656]:
DF = Labels


In [657]:
DF.columns

Index(['flight', 'log', 'date', 'pilot', 'launch_time', 'landing_time',
       'flying_time', 'launch_coord', 'landing_coord', 'landed_lz',
       'linear_dist', 'max_alt', 'total_alt_gain', 'left_perimeter', 'month',
       'year', 'timestamp', 'doy'],
      dtype='object')

In [658]:
# Dropping values earlier than 2005
DF = DF[DF['year'] > 2004]

### Clustering - Pilot ranking (by XC)

In [661]:
DF.columns

Index(['flight', 'log', 'date', 'pilot', 'launch_time', 'landing_time',
       'flying_time', 'launch_coord', 'landing_coord', 'landed_lz',
       'linear_dist', 'max_alt', 'total_alt_gain', 'left_perimeter', 'month',
       'year', 'timestamp', 'doy'],
      dtype='object')

In [696]:
from sklearn.cluster import KMeans


#grouping pilots by cluster (based on how often left perimeter)

pilot_ranking = DF.groupby('pilot')['left_perimeter'].mean()
X = pilot_ranking.as_matrix().reshape(-1, 1)

K = KMeans(n_clusters=3, random_state=0).fit(X)

clusters = K.predict(X)

In [697]:
DF['pilot_rank']=None

In [698]:
# Inserting pilot_rank
def pilot_rank(pilot):
    if pilot in cluster_0:
        return 0
    elif pilot in cluster_1:
        return 1
    elif pilot in cluster_2:
        return 2
    else:
        return 1

In [699]:
cluster_0 = set(pilot_ranking.index[clusters==0] )
cluster_1 = set(pilot_ranking.index[clusters==1] )
cluster_2 = set(pilot_ranking.index[clusters==2] )

In [701]:
DF['pilot_rank']=DF['pilot'].apply(pilot_rank)

In [ ]:
backup = DF
backup = pd.get_dummies(DF['pilot_rank'])
DF = pd.concat([DF, backup], axis=1)
DF.drop('pilot_rank',inplace=True,axis=1)

In [712]:
DF.columns

Index(['flight', 'date', 'pilot', 'launch_time', 'landing_time', 'flying_time',
       'launch_coord', 'landing_coord', 'landed_lz', 'linear_dist', 'max_alt',
       'total_alt_gain', 'left_perimeter', 'month', 'year', 'timestamp', 'doy',
       'rank_alt_0', 'rank_alt_1', 'rank_alt_2', 'overall_rank_0',
       'overall_rank_1', 'overall_rank_2', 'pilot_rank_0', 'pilot_rank_1',
       'pilot_rank_2'],
      dtype='object')

In [711]:
DF.rename(columns={0: "pilot_rank_0",1:'pilot_rank_1',2:'pilot_rank_2'},inplace=True)

### Clustering - Pilot ranking (by max alt)

In [668]:
DF.reset_index(inplace=True,drop=True)

In [669]:
#grouping pilots by cluster (based on how often left perimeter)

alt_ranking = DF.groupby('pilot')['max_alt'].mean()
X = alt_ranking.as_matrix().reshape(-1, 1)

K = KMeans(n_clusters=3, random_state=0).fit(X)

clusters2 = K.predict(X)

In [670]:
cluster2_0 = set(alt_ranking.index[clusters2==0] )
cluster2_1 = set(alt_ranking.index[clusters2==1] )
cluster2_2 = set(alt_ranking.index[clusters2==2] )

In [671]:
# Inserting alt_rank
def alt_rank(pilot):
    if pilot in cluster2_0:
        return 0
    elif pilot in cluster2_1:
        return 1
    elif pilot in cluster2_2:
        return 2


In [672]:
DF['alt_rank'] = DF['pilot'].apply(alt_rank)

In [673]:
backup = DF

In [674]:
backup = pd.get_dummies(DF['alt_rank'])

In [675]:
DF = pd.concat([DF, backup], axis=1)

In [676]:
DF.drop('alt_rank',inplace=True,axis=1)

In [677]:
DF = DF.rename(columns={0: 'rank_alt_0', 1: 'rank_alt_1',2:'rank_alt_2'})

### Clustering - Pilot ranking (by max alt and XC)

In [687]:
overall_rank = DF.groupby('pilot')['max_alt','left_perimeter'].mean()
X = overall_rank.as_matrix()
K = KMeans(n_clusters=3, random_state=0).fit(X)

clusters3 = K.predict(X)

In [689]:
cluster3_0 = set(overall_rank.index[clusters3==0] )
cluster3_1 = set(overall_rank.index[clusters3==1] )
cluster3_2 = set(overall_rank.index[clusters3==2] )

In [690]:
# Inserting alt_rank
def overall_rank(pilot):
    if pilot in cluster3_0:
        return 0
    elif pilot in cluster3_1:
        return 1
    elif pilot in cluster3_2:
        return 2
    

In [692]:
DF.columns

Index([        'flight',            'log',           'date',          'pilot',
          'launch_time',   'landing_time',    'flying_time',   'launch_coord',
        'landing_coord',      'landed_lz',    'linear_dist',        'max_alt',
       'total_alt_gain', 'left_perimeter',          'month',           'year',
            'timestamp',            'doy',     'rank_alt_0',     'rank_alt_1',
           'rank_alt_2',                0,                1,                2],
      dtype='object')

In [ ]:
backup = DF
backup = pd.get_dummies(DF['overall_rank'])
DF = pd.concat([DF, backup], axis=1)
DF['overall_rank'] = DF['pilot'].apply(overall_rank)

In [693]:
DF = DF.rename(columns={0: 'overall_rank_0', 1: 'overall_rank_1',2:'overall_rank_2'})

In [709]:
# DF.drop('overall_rank',inplace=True,axis=1)
DF.columns

Index([        'flight',           'date',          'pilot',    'launch_time',
         'landing_time',    'flying_time',   'launch_coord',  'landing_coord',
            'landed_lz',    'linear_dist',        'max_alt', 'total_alt_gain',
       'left_perimeter',          'month',           'year',      'timestamp',
                  'doy',     'rank_alt_0',     'rank_alt_1',     'rank_alt_2',
       'overall_rank_0', 'overall_rank_1', 'overall_rank_2',                0,
                      1,                2],
      dtype='object')

In [706]:
DF.drop('log',axis=1,inplace=True)

In [713]:
DF.columns

Index(['flight', 'date', 'pilot', 'launch_time', 'landing_time', 'flying_time',
       'launch_coord', 'landing_coord', 'landed_lz', 'linear_dist', 'max_alt',
       'total_alt_gain', 'left_perimeter', 'month', 'year', 'timestamp', 'doy',
       'rank_alt_0', 'rank_alt_1', 'rank_alt_2', 'overall_rank_0',
       'overall_rank_1', 'overall_rank_2', 'pilot_rank_0', 'pilot_rank_1',
       'pilot_rank_2'],
      dtype='object')

In [714]:
#Sweet way to push DF to Postgres DB


import io

from sqlalchemy import create_engine

# fail, append or replace

def write_to_table(df, db_engine, table_name, if_exists='fail'):
    string_data_io = io.StringIO()
    df.to_csv(string_data_io, sep='|', index=False)
    pd_sql_engine = pd.io.sql.pandasSQL_builder(db_engine)
    table = pd.io.sql.SQLTable(table_name, pd_sql_engine, frame=df,
                               index=False, if_exists=if_exists)
    table.create()
    string_data_io.seek(0)
    string_data_io.readline()  # remove header
    with db_engine.connect() as connection:
        with connection.connection.cursor() as cursor:
            copy_cmd = "COPY %s FROM STDIN HEADER DELIMITER '|' CSV" % table_name
            cursor.copy_expert(copy_cmd, string_data_io)
        connection.connection.commit()

In [715]:
address = 'postgresql://@localhost:5432/soaring_predictor'
engine = create_engine(address)

In [716]:
write_to_table(DF, engine, 'flight_metrics')

### Recovering DF from SQL SB

In [62]:
conn = psycopg2.connect("dbname = soaring_predictor")

In [63]:
DF = pd.read_sql('SELECT * from flight_metrics;', conn)

In [ ]:
# dates = matplotlib.dates.date2num(DF['date'])
# fig,ax = plt.subplots(figsize = (18,8))
# ax.scatter(DF['doy'], DF['max_alt'], alpha = 0.5,)
# ax.set_title('Maximum altutide [in meters]',fontsize=20)
# ax.set_ylabel('meters', fontsize=16)
# ax.set_xticks([0,90,180,270]) # choose which x locations to have ticks
# ax.set_xticklabels(['Jan','Apr',"Jul",'Oct'], fontsize=14)